In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

In [2]:
data = pd.read_csv("C:\Aniruddh\Python\Jupyter\RCC\CSV FILES\Cleaned data\RCC_cleaned_data.csv")
data

,AGE,SEX,DM,HTN,CKD,INCIDENTAL,PAIN,HEMATURIA,MASS,LOW,...,T,N,M,IVC INV,MAX DIA,APPROACH,HISTOPATHOLOGY,GRADE,MONTHS,RECURRENCE
0,32,M,0,0,0,0,1,0,0,0.0,...,2,0,1,0,8.0,2,clear cell,2.0,95,0
1,45,F,0,0,0,0,1,1,0,0.0,...,4,1,1,0,4.9,1,clear cell,2.0,95,0
2,47,M,0,0,0,1,0,0,0,0.0,...,3b,0,1,1,9.0,1,clear cell,3.0,95,0
3,58,M,0,0,0,0,0,1,0,0.0,...,3a,0,0,0,7.0,2,clear cell,2.0,95,1
4,22,M,0,0,0,0,1,0,0,0.0,...,2b,0,0,0,10.4,1,clear cell,2.0,94,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,64,M,0,0,0,0,1,0,0,0.0,...,3a,0,0,0,8.0,1,clear cell,4.0,18,0
691,49,M,0,0,0,0,0,1,0,0.0,...,2a,0,0,0,10.0,1,clear cell,3.0,18,0
692,73,M,0,1,0,0,0,1,0,0.0,...,3b,0,0,1,7.5,2,clear cell,1.0,18,0
693,65,F,0,0,0,0,1,0,0,0.0,...,3a,0,0,0,10.0,1,clear cell,1.0,18,0


In [3]:
X = data.copy()
y = X.pop('RECURRENCE')

In [4]:
X.loc[0, 'T'] = '2a'

In [5]:
X['T'].value_counts()

T
1b    178
1a    176
3a    173
2a     89
3b     26
2b     24
4      23
3c      6
Name: count, dtype: int64

In [6]:
X.loc[X[X.GRADE.isna() == True].index, 'GRADE'] = 2

In [7]:
X.GRADE.value_counts()

GRADE
2.0    290
1.0    226
3.0    123
4.0     56
Name: count, dtype: int64

In [8]:
X.loc[X[X.LOW.isna() == True].index, 'LOW'] = 0

In [9]:
X.isna().sum().sum()

0

In [10]:
T_dict = {'1a':1, '1b':2, '2a':3, '2b':4, '3a':5, '3b':6, '3c':7, '4':8}

In [11]:
sorted(X['T'].unique())

['1a', '1b', '2a', '2b', '3a', '3b', '3c', '4']

In [12]:
X['T'] = X['T'].replace(T_dict)

In [13]:
X = pd.get_dummies(X)

In [14]:
X.rename(columns = {'SITE_b/l':'SITE_BL', 'SITE_left':'SITE_LEFT', 'SITE_right':'SITE_RIGHT',
                    'HISTOPATHOLOGY_clear cell': 'HISTOPATHOLOGY_CC', 'HISTOPATHOLOGY_papillary':'HISTOPATHOLOGY_PAP'}, inplace = True)

In [15]:
X

,AGE,DM,HTN,CKD,INCIDENTAL,PAIN,HEMATURIA,MASS,LOW,FEVER,...,APPROACH,GRADE,MONTHS,SEX_F,SEX_M,SITE_BL,SITE_LEFT,SITE_RIGHT,HISTOPATHOLOGY_CC,HISTOPATHOLOGY_PAP
0,32,0,0,0,0,1,0,0,0.0,0,...,2,2.0,95,False,True,False,True,False,True,False
1,45,0,0,0,0,1,1,0,0.0,1,...,1,2.0,95,True,False,False,False,True,True,False
2,47,0,0,0,1,0,0,0,0.0,0,...,1,3.0,95,False,True,False,False,True,True,False
3,58,0,0,0,0,0,1,0,0.0,0,...,2,2.0,95,False,True,False,False,True,True,False
4,22,0,0,0,0,1,0,0,0.0,0,...,1,2.0,94,False,True,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,64,0,0,0,0,1,0,0,0.0,0,...,1,4.0,18,False,True,False,False,True,True,False
691,49,0,0,0,0,0,1,0,0.0,0,...,1,3.0,18,False,True,False,False,True,True,False
692,73,0,1,0,0,0,1,0,0.0,0,...,2,1.0,18,False,True,False,False,True,True,False
693,65,0,0,0,0,1,0,0,0.0,0,...,1,1.0,18,True,False,False,True,False,True,False


In [16]:
for col in X.columns:
    if X[col].dtype == 'bool':
        X[col] = X[col].astype(np.int64)

In [17]:
X

,AGE,DM,HTN,CKD,INCIDENTAL,PAIN,HEMATURIA,MASS,LOW,FEVER,...,APPROACH,GRADE,MONTHS,SEX_F,SEX_M,SITE_BL,SITE_LEFT,SITE_RIGHT,HISTOPATHOLOGY_CC,HISTOPATHOLOGY_PAP
0,32,0,0,0,0,1,0,0,0.0,0,...,2,2.0,95,0,1,0,1,0,1,0
1,45,0,0,0,0,1,1,0,0.0,1,...,1,2.0,95,1,0,0,0,1,1,0
2,47,0,0,0,1,0,0,0,0.0,0,...,1,3.0,95,0,1,0,0,1,1,0
3,58,0,0,0,0,0,1,0,0.0,0,...,2,2.0,95,0,1,0,0,1,1,0
4,22,0,0,0,0,1,0,0,0.0,0,...,1,2.0,94,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,64,0,0,0,0,1,0,0,0.0,0,...,1,4.0,18,0,1,0,0,1,1,0
691,49,0,0,0,0,0,1,0,0.0,0,...,1,3.0,18,0,1,0,0,1,1,0
692,73,0,1,0,0,0,1,0,0.0,0,...,2,1.0,18,0,1,0,0,1,1,0
693,65,0,0,0,0,1,0,0,0.0,0,...,1,1.0,18,1,0,0,1,0,1,0


In [18]:
one_hot_cols = []
for col in X.columns[-10:]:
    one_hot_cols.append(col)

In [19]:
one_hot_cols

['APPROACH',
 'GRADE',
 'MONTHS',
 'SEX_F',
 'SEX_M',
 'SITE_BL',
 'SITE_LEFT',
 'SITE_RIGHT',
 'HISTOPATHOLOGY_CC',
 'HISTOPATHOLOGY_PAP']

In [20]:
num_cont_cols = ['AGE', 'MAX DIA', 'MONTHS']

In [21]:
num_ord_cols = ['T', 'GRADE']

In [22]:
num_nom_cols = []
for col in X.columns[:-10]:
    if col not in num_cont_cols and col not in num_ord_cols:
        num_nom_cols.append(col)

In [23]:
num_nom_cols

['DM',
 'HTN',
 'CKD',
 'INCIDENTAL',
 'PAIN',
 'HEMATURIA',
 'MASS',
 'LOW',
 'FEVER',
 'N',
 'M',
 'IVC INV']

In [24]:
X = pd.get_dummies(X, columns = ['APPROACH'], dtype = np.int64)

In [25]:
X['RECURRENCE'] = y
X

,AGE,DM,HTN,CKD,INCIDENTAL,PAIN,HEMATURIA,MASS,LOW,FEVER,...,SEX_M,SITE_BL,SITE_LEFT,SITE_RIGHT,HISTOPATHOLOGY_CC,HISTOPATHOLOGY_PAP,APPROACH_1,APPROACH_2,APPROACH_3,RECURRENCE
0,32,0,0,0,0,1,0,0,0.0,0,...,1,0,1,0,1,0,0,1,0,0
1,45,0,0,0,0,1,1,0,0.0,1,...,0,0,0,1,1,0,1,0,0,0
2,47,0,0,0,1,0,0,0,0.0,0,...,1,0,0,1,1,0,1,0,0,0
3,58,0,0,0,0,0,1,0,0.0,0,...,1,0,0,1,1,0,0,1,0,1
4,22,0,0,0,0,1,0,0,0.0,0,...,1,0,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,64,0,0,0,0,1,0,0,0.0,0,...,1,0,0,1,1,0,1,0,0,0
691,49,0,0,0,0,0,1,0,0.0,0,...,1,0,0,1,1,0,1,0,0,0
692,73,0,1,0,0,0,1,0,0.0,0,...,1,0,0,1,1,0,0,1,0,0
693,65,0,0,0,0,1,0,0,0.0,0,...,0,0,1,0,1,0,1,0,0,0


In [26]:
X_majority = X[X['RECURRENCE'] == 0]
X_minority = X[X['RECURRENCE'] == 1]

In [27]:
X_minority_oversampled = resample(X_minority, replace = True, n_samples = len(X_majority), random_state = 42)

In [28]:
X_minority_oversampled

,AGE,DM,HTN,CKD,INCIDENTAL,PAIN,HEMATURIA,MASS,LOW,FEVER,...,SEX_M,SITE_BL,SITE_LEFT,SITE_RIGHT,HISTOPATHOLOGY_CC,HISTOPATHOLOGY_PAP,APPROACH_1,APPROACH_2,APPROACH_3,RECURRENCE
418,53,0,0,0,0,0,1,0,0.0,0,...,1,0,1,0,1,0,1,0,0,1
143,61,1,0,0,1,0,0,0,0.0,0,...,1,0,0,1,1,0,0,0,1,1
504,50,0,0,0,0,0,1,0,0.0,0,...,0,0,0,1,1,0,1,0,0,1
452,60,0,0,0,1,0,0,0,0.0,0,...,1,0,0,1,1,0,0,1,0,1
225,60,0,0,0,0,0,1,0,0.0,0,...,0,0,0,1,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,51,0,0,0,0,1,0,0,0.0,0,...,1,0,0,1,1,0,1,0,0,1
387,56,0,0,0,0,0,1,0,0.0,0,...,0,0,0,1,1,0,1,0,0,1
322,60,0,0,0,1,0,0,0,0.0,0,...,1,0,1,0,1,0,0,1,0,1
387,56,0,0,0,0,0,1,0,0.0,0,...,0,0,0,1,1,0,1,0,0,1


In [29]:
# Combine the majority class with the oversampled minority class
X_oversampled = pd.concat([X_majority, X_minority_oversampled])

# Shuffle the dataset to ensure randomness
X_oversampled = X_oversampled.sample(frac=1, random_state=42)

In [30]:
X_oversampled.RECURRENCE.value_counts()

RECURRENCE
0    608
1    608
Name: count, dtype: int64

In [31]:
y_oversampled = X_oversampled.pop("RECURRENCE")

In [32]:
X_smote = X.copy()

In [33]:
smote = SMOTE(random_state = 42)
y_smote = X_smote.pop('RECURRENCE')

In [34]:
X_train_smote, X_test_smote, y_train_smote, y_test_smote = train_test_split(X_smote, y_smote, test_size = 0.2, random_state = 42)

In [35]:
X_train_resampled_smote, y_train_resampled_smote = smote.fit_resample(X_train_smote, y_train_smote)

In [36]:
scaler = StandardScaler()
X_ov_train, X_ov_test, y_ov_train, y_ov_test = train_test_split(X_oversampled, y_oversampled, test_size = 0.25, random_state = 42)

In [37]:
X_ov_train[num_cont_cols] = scaler.fit_transform(X_ov_train[num_cont_cols])
X_ov_test[num_cont_cols] = scaler.transform(X_ov_test[num_cont_cols])

In [38]:
X_ov_train.shape

(912, 27)

In [39]:
y_ov_train.value_counts()

RECURRENCE
1    463
0    449
Name: count, dtype: int64

In [40]:
y_ov_test.value_counts()

RECURRENCE
0    159
1    145
Name: count, dtype: int64

In [41]:
X_ov_test.shape

(304, 27)

In [42]:
model = RandomForestClassifier(random_state=42)
model.fit(X_ov_train, y_ov_train)

RandomForestClassifier(random_state=42)

In [43]:
y_pred = model.predict(X_ov_test)
accuracy = accuracy_score(y_ov_test, y_pred)
report = classification_report(y_ov_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.9802631578947368
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.96      0.98       159
           1       0.96      1.00      0.98       145

    accuracy                           0.98       304
   macro avg       0.98      0.98      0.98       304
weighted avg       0.98      0.98      0.98       304



In [44]:
xmodel = RandomForestClassifier(random_state=42)
xmodel.fit(X_ov_train, y_ov_train)
y_pred = xmodel.predict(X_ov_test)
accuracy = accuracy_score(y_ov_test, y_pred)
report = classification_report(y_ov_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.9802631578947368
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.96      0.98       159
           1       0.96      1.00      0.98       145

    accuracy                           0.98       304
   macro avg       0.98      0.98      0.98       304
weighted avg       0.98      0.98      0.98       304



In [45]:
y_ov_test.value_counts()

RECURRENCE
0    159
1    145
Name: count, dtype: int64

In [46]:
new_scaler = StandardScaler()
X[num_cont_cols] = new_scaler.fit_transform(X[num_cont_cols])
X

,AGE,DM,HTN,CKD,INCIDENTAL,PAIN,HEMATURIA,MASS,LOW,FEVER,...,SEX_M,SITE_BL,SITE_LEFT,SITE_RIGHT,HISTOPATHOLOGY_CC,HISTOPATHOLOGY_PAP,APPROACH_1,APPROACH_2,APPROACH_3,RECURRENCE
0,-1.718760,0,0,0,0,1,0,0,0.0,0,...,1,0,1,0,1,0,0,1,0,0
1,-0.707033,0,0,0,0,1,1,0,0.0,1,...,0,0,0,1,1,0,1,0,0,0
2,-0.551383,0,0,0,1,0,0,0,0.0,0,...,1,0,0,1,1,0,1,0,0,0
3,0.304694,0,0,0,0,0,1,0,0.0,0,...,1,0,0,1,1,0,0,1,0,1
4,-2.497011,0,0,0,0,1,0,0,0.0,0,...,1,0,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,0.771645,0,0,0,0,1,0,0,0.0,0,...,1,0,0,1,1,0,1,0,0,0
691,-0.395732,0,0,0,0,0,1,0,0.0,0,...,1,0,0,1,1,0,1,0,0,0
692,1.472071,0,1,0,0,0,1,0,0.0,0,...,1,0,0,1,1,0,0,1,0,0
693,0.849470,0,0,0,0,1,0,0,0.0,0,...,0,0,1,0,1,0,1,0,0,0


In [47]:
y = X.pop('RECURRENCE')

In [52]:
X

,AGE,DM,HTN,CKD,INCIDENTAL,PAIN,HEMATURIA,MASS,LOW,FEVER,...,SEX_F,SEX_M,SITE_BL,SITE_LEFT,SITE_RIGHT,HISTOPATHOLOGY_CC,HISTOPATHOLOGY_PAP,APPROACH_1,APPROACH_2,APPROACH_3
0,-1.718760,0,0,0,0,1,0,0,0.0,0,...,0,1,0,1,0,1,0,0,1,0
1,-0.707033,0,0,0,0,1,1,0,0.0,1,...,1,0,0,0,1,1,0,1,0,0
2,-0.551383,0,0,0,1,0,0,0,0.0,0,...,0,1,0,0,1,1,0,1,0,0
3,0.304694,0,0,0,0,0,1,0,0.0,0,...,0,1,0,0,1,1,0,0,1,0
4,-2.497011,0,0,0,0,1,0,0,0.0,0,...,0,1,0,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,0.771645,0,0,0,0,1,0,0,0.0,0,...,0,1,0,0,1,1,0,1,0,0
691,-0.395732,0,0,0,0,0,1,0,0.0,0,...,0,1,0,0,1,1,0,1,0,0
692,1.472071,0,1,0,0,0,1,0,0.0,0,...,0,1,0,0,1,1,0,0,1,0
693,0.849470,0,0,0,0,1,0,0,0.0,0,...,1,0,0,1,0,1,0,1,0,0


In [51]:
ymodel = RandomForestClassifier(random_state=42)
ymodel.fit(X, y)
y_pred = ymodel.predict(X)
accuracy = accuracy_score(y, y_pred)
report = classification_report(y, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       608
           1       1.00      1.00      1.00        87

    accuracy                           1.00       695
   macro avg       1.00      1.00      1.00       695
weighted avg       1.00      1.00      1.00       695



In [49]:
X

,AGE,DM,HTN,CKD,INCIDENTAL,PAIN,HEMATURIA,MASS,LOW,FEVER,...,SEX_F,SEX_M,SITE_BL,SITE_LEFT,SITE_RIGHT,HISTOPATHOLOGY_CC,HISTOPATHOLOGY_PAP,APPROACH_1,APPROACH_2,APPROACH_3
0,-1.718760,0,0,0,0,1,0,0,0.0,0,...,0,1,0,1,0,1,0,0,1,0
1,-0.707033,0,0,0,0,1,1,0,0.0,1,...,1,0,0,0,1,1,0,1,0,0
2,-0.551383,0,0,0,1,0,0,0,0.0,0,...,0,1,0,0,1,1,0,1,0,0
3,0.304694,0,0,0,0,0,1,0,0.0,0,...,0,1,0,0,1,1,0,0,1,0
4,-2.497011,0,0,0,0,1,0,0,0.0,0,...,0,1,0,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,0.771645,0,0,0,0,1,0,0,0.0,0,...,0,1,0,0,1,1,0,1,0,0
691,-0.395732,0,0,0,0,0,1,0,0.0,0,...,0,1,0,0,1,1,0,1,0,0
692,1.472071,0,1,0,0,0,1,0,0.0,0,...,0,1,0,0,1,1,0,0,1,0
693,0.849470,0,0,0,0,1,0,0,0.0,0,...,1,0,0,1,0,1,0,1,0,0
